```
This software is a part of GPU Ocean.

Copyright (C) 2019  SINTEF Digital

This notebook is used to create figures that illustrates how the 
optimal proposal pull (earlier called the Kalman gain term) acts on 
a particle.

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <http://www.gnu.org/licenses/>.
```

# Optimal proposal pull figures

Plan:
- Initiate an ensemble with 5 members with zeros + 100*q + 10*q, and make a keepInfoPlot-plot to see that this ensemble can be used. Use a fixed seed and numpy random numbers on the host to make it reproducible.
- Size should be small enough so that only one drifter is sufficient. Should use interpolation factor 5, and we get an influence radius of 45 grid cells. So we need grid that is of size 120x120.
- Drifter should be in position (x,y) = (40, 70).


## Set environment

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import numpy as np
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import animation, rc
from scipy.special import lambertw

import pycuda.driver as cuda

import os
import sys
import datetime
from importlib import reload

sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '../../')))

#Set large figure sizes
rc('figure', figsize=(16.0, 12.0))
rc('animation', html='html5')
rc('font',**{'family':'sans-serif','sans-serif':['Helvetica'], 'size':16})
rc('text', usetex=True)
rc('xtick', labelsize=14)     
rc('ytick', labelsize=14)

matplotlib.rcParams['contour.negative_linestyle'] = 'solid'


#Import our simulator
from SWESimulators import CDKLM16, PlotHelper, Common, IPythonMagic, config

from SWESimulators import BathymetryAndICs as BC
from SWESimulators import OceanStateNoise
from SWESimulators import OceanNoiseEnsemble
from SWESimulators import BaseOceanStateEnsemble
from SWESimulators import DataAssimilationUtils as dautils
from SWESimulators import IEWPFOcean

#np.random.seed(8) # 8 was fine, but velocities the wrong way...
#np.random.seed(11) # 11 was fine also, but innovation straight north...
np.random.seed(25) # 11 was fine also, but innovation straight north...

In [ ]:
%cuda_context_handler gpu_ctx
%setup_logging --out iewpf_2stage_gpu.log --file_level $config.GPUOceanLoggerLevels.IEWPF_DEBUG.value

In [ ]:
#Create output directory for images
imgdir='optimal_proposal_pull_' + datetime.datetime.now().strftime("%Y_%m_%d-%H_%M_%S")
os.makedirs(imgdir)
imgdir = imgdir + "/"
print("Saving figures to " + imgdir)

# Testing two-stage IEWPF

Here, we make a test of the entire two-stage IEWPF algorithm applied to a suitable test case.

In [ ]:
# DEFINE PARAMETERS

sim_args = {
    "gpu_ctx": gpu_ctx,
    "nx": 120, "ny": 120,
    #"nx": 15, "ny": 15,
    "dx": 4.0, "dy": 4.0,
    "dt": 0.05,
    "g": 9.81,
    "f": 0.05,
    "coriolis_beta": 0.0,
    "r": 0.0,
    "rk_order": 2,
    "small_scale_perturbation_amplitude": None,
    "write_netcdf": False, 
    "small_scale_perturbation": True,
    #"small_scale_perturbation_interpolation_factor": 1,
    "small_scale_perturbation_interpolation_factor": 5,
    "boundary_conditions": Common.BoundaryConditions(2,2,2,2)
}

ghosts = np.array([2,2,2,2]) # north, east, south, west
validDomain = np.array([2,2,2,2])


dataShape = (sim_args["ny"] + ghosts[0]+ghosts[2], 
             sim_args["nx"] + ghosts[1]+ghosts[3])
dataShapeHi = (sim_args["ny"] + ghosts[0]+ghosts[2]+1, 
             sim_args["nx"] + ghosts[1]+ghosts[3]+1)

sim_args["eta0"] = np.zeros(dataShape, dtype=np.float32, order='C');
sim_args["hv0"] = np.zeros(dataShape, dtype=np.float32, order='C');
sim_args["hu0"] = np.zeros(dataShape, dtype=np.float32, order='C');
waterDepth = 10.0
sim_args["H"] = np.ones(dataShapeHi, dtype=np.float32, order='C')*waterDepth


if 'sim' in globals(): 
    sim.cleanUp()
if 'ensemble' in globals():
    ensemble.cleanUp()
if 'iewpfOcean' in globals():
    iewpfOcean.cleanUp()
    
sim_args["small_scale_perturbation_amplitude"] = np.sqrt(sim_args["dt"])*0.5*sim_args["f"]/(sim_args["g"]*waterDepth)
sim_args["small_scale_perturbation_amplitude"] *= sim_args["small_scale_perturbation_interpolation_factor"]
print ("q0: ", sim_args["small_scale_perturbation_amplitude"])
print ("[f, g, H, dt]", [sim_args["f"], sim_args["g"], waterDepth, sim_args["dt"]])
print ("(nx, ny): ", (sim_args["nx"], sim_args["ny"]))

reload(CDKLM16)
reload(BaseOceanStateEnsemble)
reload(OceanNoiseEnsemble)
reload(PlotHelper)
reload(dautils)
reload(IEWPFOcean)

sim = CDKLM16.CDKLM16(**sim_args)

# Initial perturbation
rand_nx, rand_ny = sim.small_scale_model_error.rand_nx, sim.small_scale_model_error.rand_ny
init_rand = np.random.normal(size=(rand_ny, rand_nx))
sim.small_scale_model_error.random_numbers.upload(sim.gpu_stream, init_rand)
sim.small_scale_model_error.perturbSim(sim, q0_scale=100, update_random_field=False)
    
eta_init, hu_init, hv_init = sim.download(True)
    
ensemble_args = {
    "gpu_ctx": gpu_ctx,
    "numParticles": 5,
    "num_drifters": 1,
    "sim": sim,
    "observation_type": dautils.ObservationType.DirectUnderlyingFlow,
    "observation_variance": 0.015**2 #0.02**2
}

ensemble = OceanNoiseEnsemble.OceanNoiseEnsemble(**ensemble_args)

max_dt = ensemble.findLargestPossibleTimeStep()
print ("max_dt: ", max_dt)

iewpfOcean = IEWPFOcean.IEWPFOcean(ensemble, debug=False, show_errors=True)

print ("Ready!")
#ensemble.plotEnsemble()

In [ ]:
# Set drifter position
ensemble.setTrueDrifterPositions([[75.5*ensemble.dx,40.5*ensemble.dy]])
#ensemble.setTrueDrifterPositions([[35.5*ensemble.dx,35.5*ensemble.dy]])

for particle in ensemble.particles:
    perturbation = np.random.normal(size=(rand_ny, rand_nx))
    particle.small_scale_model_error.random_numbers.upload(particle.gpu_stream, perturbation)
    particle.small_scale_model_error.perturbSim(particle, q0_scale=30, update_random_field=False)
    
ensemble.step(ensemble.dt)

In [ ]:
ensemble.plotVelocityInfo("Prior to DA")
print("hei")

In [ ]:
print(ensemble.getInnovations())

In [ ]:
ideal_particle=3

In [ ]:
ensemble.plotEnsemble()

In [ ]:
def showMatrices3(eta, hu, hv, title, drifterPositions=None, ensemble=None):
    num_cols = 3
    min_velocity = min(np.min(hu), np.min(hv))
    max_velocity = max(np.max(hu), np.max(hv))
    
    fig = plt.figure(figsize=(12,4))
    ax = plt.subplot(1,num_cols,1)
    plt.imshow(eta, origin="lower", interpolation="None")
    if drifterPositions is not None:
        _markDriftersInImshow(ax, drifterPositions, ensemble)
    
    ax = plt.subplot(1,num_cols,2)
    plt.imshow(hu, origin="lower", interpolation="None", 
               vmin=min_velocity, vmax=max_velocity)
    if drifterPositions is not None:
        _markDriftersInImshow(ax, drifterPositions, ensemble)

    ax = plt.subplot(1, num_cols, 3)
    plt.imshow(hv, origin="lower", interpolation="None", 
           vmin=min_velocity, vmax=max_velocity)
    if drifterPositions is not None:
        _markDriftersInImshow(ax, drifterPositions, ensemble)


def imshow(data, interpolation="None", title=None, figsize=(4,4), interior=False):
    fig = plt.figure(figsize=figsize)
    
    if interior:
        im = plt.imshow(data[2:-2,2:-2], interpolation=interpolation, origin='lower')
    else:
        im = plt.imshow(data, interpolation=interpolation, origin='lower')
    
    plt.colorbar()
    if title is not None:
        plt.title(title, fontsize=15)
    ny, nx = data.shape
    axis_label_text = r'$\Omega^R$'
    if ny == ensemble.ny:
        axis_label_text = r'$\Omega^M$'
    axis_label = plt.text(-nx/6, -ny/6, axis_label_text, fontsize=15)

        
        
def imshow3(eta, hu, hv, interpolation="None", title=None, figsize=(13,4), interior=False, infotext=None):

    ny, nx = eta.shape
    axis_label_text = r'$\Omega^R$'
    if ny == ensemble.ny:
        axis_label_text = r'$\Omega^M$'
    
    fig = None
    axs = None
    if infotext is None:
        fig, axs = plt.subplots(1,3, figsize=figsize)
    else:
        fig, axs = plt.subplots(1,4, figsize=(figsize[0]*1.33, figsize[1]))
    
    range_eta = np.max(np.abs(eta))
    range_huv = max(np.max(np.abs(hu)), np.max(np.abs(hv)))
    
    if interior:
        eta_im = axs[0].imshow(eta[2:-2,2:-2], interpolation=interpolation, origin='lower', vmin=-range_eta, vmax=range_eta)
    else:
        eta_im = axs[0].imshow(eta, interpolation=interpolation, origin='lower', vmin=-range_eta, vmax=range_eta)
    axs[0].set_title("$\eta$", fontsize=14)
    plt.colorbar(eta_im, ax=axs[0])
    axis_label = axs[0].text(-nx/6, -ny/6, axis_label_text, fontsize=15)
    
    if interior:
        hu_im = axs[1].imshow(hu[2:-2,2:-2], interpolation=interpolation, origin='lower', vmin=-range_huv, vmax=range_huv)
    else:
        hu_im = axs[1].imshow(hu, interpolation=interpolation, origin='lower', vmin=-range_huv, vmax=range_huv)
    axs[1].set_title("$hu$", fontsize=14)
    plt.colorbar(hu_im, ax=axs[1])
    axis_label = axs[1].text(-nx/6, -ny/6, axis_label_text, fontsize=15)
    
    if interior:
        hv_im = axs[2].imshow(hv[2:-2,2:-2], interpolation=interpolation, origin='lower', vmin=-range_huv, vmax=range_huv)
    else:
        hv_im = axs[2].imshow(hv, interpolation=interpolation, origin='lower', vmin=-range_huv, vmax=range_huv)
    axs[2].set_title("$hv$", fontsize=14)
    plt.colorbar(hv_im, ax=axs[2])
    axis_label = axs[2].text(-nx/6, -ny/6, axis_label_text, fontsize=15)

    if infotext is not None:
        axs[3].text(0.5, 0.5, infotext, ha='center')

    plt.subplots_adjust(top=0.88)                        
    if title is not None:
        if infotext is None:
            plt.suptitle(title,  y=1.00, fontsize=16)
        else: 
            plt.suptitle(title, x=3.0/8.0,  y=1.00, fontsize=16)
            
    plt.tight_layout()
    return fig
        
def createInnovationPlot(ensemble, particle_id, S):
    drifter_position = ensemble.observeTrueDrifters()[0,:]
    true_velocity = ensemble.observeTrueState()[0, 2:]
    all_particle_velocities = ensemble.observeParticles()
    particle_velocity = ensemble.observeParticles()[particle_id,0,:]
    innovation = ensemble.getInnovations()[particle_id,0,:]
    
    print("drifter_position", drifter_position)
    print("true_velocity", true_velocity)
    #print("all_particle_velocities", all_particle_velocities)    
    print("particle_velocity", particle_velocity)
    print("innovation: ", innovation)
    
    cell_id_x = int(np.floor(drifter_position[0]/ensemble.dx))
    cell_id_y = int(np.floor(drifter_position[1]/ensemble.dy))
    print("cell_id: ", (cell_id_x, cell_id_y))
    
    fig = plt.figure(figsize=(4,4))
    ax = plt.subplot(111)
    ax.imshow(np.zeros((ensemble.ny, ensemble.nx))*np.nan, origin="lower")

    print("S: ", S)
    print("S*d: ", np.dot(S, innovation))#S*ensemble.getInnovations()[particle_id,:,:].transpose)

    circ = matplotlib.patches.Circle((cell_id_x, cell_id_y), 2, fill=False)
    ax.add_patch(circ)
    
    # Add observed particle and truth
    arrow_scale = 20
    particle_arrow = matplotlib.pyplot.arrow(cell_id_x, cell_id_y, 
                                             particle_velocity[0]*arrow_scale, 
                                             particle_velocity[1]*arrow_scale, \
                                             color='xkcd:electric pink', width=0.7)
    ax.add_patch(particle_arrow)
    
    truth_arrow = matplotlib.pyplot.arrow(cell_id_x, cell_id_y, 
                                          true_velocity[0]*arrow_scale, 
                                          true_velocity[1]*arrow_scale, \
                                          color='xkcd:azure', width=0.7)
    ax.add_patch(truth_arrow)
    
    innovation_arrow = matplotlib.pyplot.arrow(cell_id_x, #+particle_velocity[0]*arrow_scale, 
                                               cell_id_y, #+particle_velocity[1]*arrow_scale, 
                                               innovation[0]*arrow_scale, 
                                               innovation[1]*arrow_scale, 
                                               linestyle=':',
                                               color='xkcd:dark slate blue', width=0.7)
    ax.add_patch(innovation_arrow)
    
    text_particle = plt.text(cell_id_x+particle_velocity[0]*arrow_scale+3, 
                             cell_id_y+particle_velocity[1]*arrow_scale, 
                             r'$H(\psi^n)$', fontsize=15)
    text_observation = plt.text(cell_id_x+true_velocity[0]*arrow_scale-13, 
                                cell_id_y+true_velocity[1]*arrow_scale, 
                                r'$y^n$', fontsize=15)
    text_innovation = plt.text(cell_id_x+innovation[0]*arrow_scale-48, 
                               cell_id_y+innovation[1]*arrow_scale-12, 
                               r'$d^n = y^n - H(\psi^n)$', fontsize=15)
    #ax.annotate('', xy=(1.2, 0.3), xycoords='axes fraction', xytext=(1.1, 0.4), 
    #        arrowprops=dict(arrowstyle="->", color='green'))
    #fig.canvas.draw()

    axis_label = plt.text(-ensemble.nx/7, -ensemble.ny/7, r'$\Omega^M$', fontsize=15)

    plt.title('Observation, particle state and innovation', fontsize=15)
    
    return ax
            


In [ ]:
ax = createInnovationPlot(ensemble, ideal_particle, iewpfOcean.S_host)
plt.savefig(imgdir + "1_vectors.pdf", bbox_inches="tight", format='pdf')

In [ ]:
# Innovation times S
S = iewpfOcean.S_host
innovation = ensemble.getInnovations()[ideal_particle,0,:]

Sd = np.dot(S, innovation)
print("S*d: ", Sd)#S*ensemble.getInnovations()[particle_id,:,:].transpose)

drifter_position = ensemble.observeTrueDrifters()[0,:]
coarse_dx = ensemble.particles[ideal_particle].small_scale_model_error.coarse_dx
coarse_dy = ensemble.particles[ideal_particle].small_scale_model_error.coarse_dy
coarse_cell_id_x = int(np.floor(drifter_position[0]/coarse_dx))
coarse_cell_id_y = int(np.floor(drifter_position[1]/coarse_dy))
print("cell_id: ", (coarse_cell_id_x, coarse_cell_id_y))

rand_nx, rand_ny = sim.small_scale_model_error.rand_nx, sim.small_scale_model_error.rand_ny
zero_buffer = np.zeros((rand_ny, rand_nx), dtype=np.float32)

IHSd_eta = zero_buffer.copy()
IHSd_hu  = zero_buffer.copy()
IHSd_hv  = zero_buffer.copy()

IHSd_hu[coarse_cell_id_y, coarse_cell_id_x] = Sd[0,0]
IHSd_hv[coarse_cell_id_y, coarse_cell_id_x] = Sd[0,1]

imshow3(IHSd_eta, IHSd_hu, IHSd_hv,
        title=r'$I^T_{\Omega}  H^T S d$')

plt.savefig(imgdir + "2_innovation_in_statespace.pdf", bbox_inches="tight", format='pdf')

print("yeah...")

In [ ]:
QgbTIHSd  = zero_buffer.copy()

geoBalanceConst = iewpfOcean.geoBalanceConst

j = coarse_cell_id_y
i = coarse_cell_id_x

QgbTIHSd[j+1, i  ] = -Sd[0,0]*geoBalanceConst/coarse_dy
QgbTIHSd[j-1, i  ] =  Sd[0,0]*geoBalanceConst/coarse_dy
QgbTIHSd[j  , i+1] =  Sd[0,1]*geoBalanceConst/coarse_dx
QgbTIHSd[j  , i-1] = -Sd[0,1]*geoBalanceConst/coarse_dx

#// the x-component of the innovation spreads to north and south
#shared_huhv[0] = -e_x_*geoBalanceConst_/dy_; // north 
#shared_huhv[2] =  e_x_*geoBalanceConst_/dy_; // south
#// the y-component of the innovation spreads to east and west
#shared_huhv[1] =  e_y_*geoBalanceConst_/dx_; // east
#shared_huhv[3] = -e_y_*geoBalanceConst_/dx_; // west


imshow(QgbTIHSd, title=r'$Q^{1/2, T}_{GB} I^T_{\Omega}  H^T S d$')

plt.savefig(imgdir + "3_innovation_five_point.pdf", bbox_inches="tight", format='pdf')

In [ ]:

iewpfOcean.setNoiseBufferToZero(ensemble.particles[ideal_particle])
        
local_innovation = innovation
observed_drifter_position = drifter_position

cell_id_x = np.int32(int(np.floor(observed_drifter_position[0]/iewpfOcean.dx)))
cell_id_y = np.int32(int(np.floor(observed_drifter_position[1]/iewpfOcean.dy)))
coarse_cell_id_x = np.int32(int(np.floor(observed_drifter_position[0]/iewpfOcean.coarse_dx)))
coarse_cell_id_y = np.int32(int(np.floor(observed_drifter_position[1]/iewpfOcean.coarse_dy)))

# 1) Solve linear problem
e = np.dot(iewpfOcean.S_host, local_innovation)

iewpfOcean.halfTheKalmanGainKernel.prepared_async_call(iewpfOcean.global_size_Kalman,
                                                 iewpfOcean.local_size_Kalman,
                                                 ensemble.particles[ideal_particle].gpu_stream,
                                                 iewpfOcean.coarse_nx, iewpfOcean.coarse_ny, 
                                                 iewpfOcean.coarse_dx, iewpfOcean.coarse_dy,
                                                 iewpfOcean.soar_q0, iewpfOcean.soar_L,
                                                 coarse_cell_id_x, coarse_cell_id_y,
                                                 iewpfOcean.geoBalanceConst,
                                                 np.float32(e[0,0]), np.float32(e[0,1]),
                                                 ensemble.particles[ideal_particle].small_scale_model_error.random_numbers.data.gpudata,
                                                 ensemble.particles[ideal_particle].small_scale_model_error.random_numbers.pitch)

half_kalman_gain = ensemble.particles[ideal_particle].small_scale_model_error.getRandomNumbers()


In [ ]:
imshow(half_kalman_gain, title=r'$Q^{1/2}_{SOAR}Q^{1/2, T}_{GB} I^T_{\Omega}  H^T S d$')

plt.savefig(imgdir + "4_soar1.pdf", bbox_inches="tight", format='pdf')

In [ ]:
## Store the state prior to adding 
## the Kalman gain, so that we 
## can subtract it for plotting the
## contribution.

pre_eta, pre_hu, pre_hv = ensemble.particles[ideal_particle].download(interior_domain_only=True)

In [ ]:
iewpfOcean.addKalmanGain(ensemble.particles[ideal_particle], 
                         ensemble.observeTrueDrifters(), 
                         ensemble.getInnovations()[ideal_particle])

In [ ]:
half_kalman_gain_soar = ensemble.particles[ideal_particle].small_scale_model_error.getCoarseBuffer()
imshow(half_kalman_gain_soar, 
       title=r'$Q^{1/2}_{SOAR}Q^{1/2}_{SOAR}Q^{1/2, T}_{GB} I^T_{\Omega}  H^T S d$')
plt.savefig(imgdir + "5_soar2.pdf", bbox_inches="tight", format='pdf')

In [ ]:
post_eta, post_hu, post_hv = ensemble.particles[ideal_particle].download(interior_domain_only=True)
kalman_eta = post_eta - pre_eta
kalman_hu  = post_hu  - pre_hu
kalman_hv  = post_hv  - pre_hv

In [ ]:


imshow(kalman_eta,
       title=r'$I_{\Omega}Q^{1/2}_{SOAR}Q^{1/2}_{SOAR}Q^{1/2, T}_{GB} I^T_{\Omega}  H^T S d$')
plt.savefig(imgdir + "6_interpolation.pdf", bbox_inches="tight", format='pdf')

In [ ]:
imshow3(kalman_eta, kalman_hu, kalman_hv,
        title=r'$Q^{1/2}_{GB}I_{\Omega}Q^{1/2}_{SOAR}Q^{1/2}_{SOAR}Q^{1/2, T}_{GB} I^T_{\Omega}  H^T S d$')
print("hei")
#plt.suptitle(r'$Q^{1/2}_{GB}I_{\Omega}Q^{1/2}_{SOAR}Q^{1/2}_{SOAR}Q^{1/2, T}_{GB} I^T_{\Omega}  H^T S d$', 
#             y=1.0, fontsize=14)
plt.tight_layout()
plt.savefig(imgdir + "7_thepull.pdf", bbox_inches="tight", format='pdf')

In [ ]:
plt.plot(post_eta[40, :])
plt.plot(post_eta[37, :])
plt.plot(post_eta[45, :])
plt.plot(post_eta[43, :])
plt.plot(post_eta[50, :])
plt.plot(post_eta[34, :])

In [ ]:
plt.plot(post_hu[40, :])
plt.plot(post_hu[37, :])
plt.plot(post_hu[45, :])
plt.plot(post_hu[43, :])
plt.plot(post_hu[50, :])
plt.plot(post_hu[34, :])

In [ ]:
plt.plot(post_hv[40, :])
plt.plot(post_hv[37, :])
plt.plot(post_hv[45, :])
plt.plot(post_hv[43, :])
plt.plot(post_hv[50, :])
plt.plot(post_hv[34, :])

In [ ]:
plt.plot(post_eta[:,70])
plt.plot(post_eta[:,67])
plt.plot(post_eta[:,75])
plt.plot(post_eta[:,73])
plt.plot(post_eta[:,80])
plt.plot(post_eta[:,64])

In [ ]:
ax = createInnovationPlot(ensemble, ideal_particle, iewpfOcean.S_host)
plt.savefig(imgdir + "8_post_kalman_state.pdf", format='pdf')

In [ ]:
sdfgg

## Old stuff

In [ ]:
# Utility plotter function
def imshow3(eta, hu, hv, interpolation="None", title=None, interior=False):
    fig, axs = plt.subplots(1,3, figsize=(12,4))
    
    if interior:
        eta_im = axs[0].imshow(eta[2:-2,2:-2], interpolation=interpolation, origin='lower')
    else:
        eta_im = axs[0].imshow(eta, interpolation=interpolation, origin='lower')
    axs[0].set_title("eta")
    plt.colorbar(eta_im, ax=axs[0])
    
    if interior:
        hu_im = axs[1].imshow(hu[2:-2,2:-2], interpolation=interpolation, origin='lower')
    else:
        hu_im = axs[1].imshow(hu, interpolation=interpolation, origin='lower')
    axs[1].set_title("hu")
    plt.colorbar(hu_im, ax=axs[1])

    if interior:
        hv_im = axs[2].imshow(hv[2:-2,2:-2], interpolation=interpolation, origin='lower')
    else:
        hv_im = axs[2].imshow(hv, interpolation=interpolation, origin='lower')
    axs[2].set_title("hv")
    plt.colorbar(hv_im, ax=axs[2])

    if title is not None:
        plt.suptitle(title)
    plt.tight_layout()

In [ ]:
if 'sim' in globals():
    sim.cleanUp()
if 'ensemble' in globals():
    ensemble.cleanUp()
if 'iewpfOcean' in globals():
    iewpfOcean.cleanUp()


reload(OceanStateNoise)
reload(CDKLM16)
reload(BaseOceanStateEnsemble)
reload(OceanNoiseEnsemble)
reload(PlotHelper)
reload(dautils)
reload(IEWPFOcean)


# Create a new ensemble with an initial perturbation
sim = CDKLM16.CDKLM16(**sim_args)
sim.perturbState(q0_scale=100)

ensemble_args["numParticles"] = 5
ensemble_args["num_drifters"] = 3
ensemble_args["sim"] = sim
ensemble = OceanNoiseEnsemble.OceanNoiseEnsemble(**ensemble_args)

iewpfOcean = IEWPFOcean.IEWPFOcean(ensemble, debug=False, show_errors=True)

# Ensuring that the particles behave a bit different from each others
ensemble.step(100*ensemble.dt)

# Set the state of particle 0 to zero 
zeros = np.zeros((ensemble.ny+4, ensemble.nx+4), dtype=np.float32)
ensemble.particles[0].gpu_data.h0.upload(ensemble.particles[0].gpu_stream, zeros)
ensemble.particles[0].gpu_data.hu0.upload(ensemble.particles[0].gpu_stream, zeros)
ensemble.particles[0].gpu_data.hv0.upload(ensemble.particles[0].gpu_stream, zeros)


# Plot
#ensemble.plotEnsemble(num_particles=2)
print("ready")

The initial steps of IEWPF before looping through particles

In [ ]:
# Observe true drifter positions and get innovations
observed_drifter_positions = ensemble.observeTrueDrifters()
print("observed drifter positions: ", observed_drifter_positions)

# Innovations
innovations = ensemble.getInnovations()
print("Innovations: ", innovations)

# Get the weights before resampling - all equal, so we only make a scalar (instead of a vector)
w_rest = -np.log(1.0/ensemble.getNumParticles())*np.ones(ensemble.getNumParticles())
print("w_rest: ", w_rest)

phi_array     = np.zeros(ensemble.getNumParticles())
zeta_array = np.zeros(ensemble.getNumParticles())
gamma_array   = np.zeros(ensemble.getNumParticles())

Now we start working with the individual particles. First, we add **the Kalman gain**

In [ ]:
# Loop step 1: Pull particles towards observation by adding a Kalman gain term
#     Also, we find phi within this function
phi_array[0] = iewpfOcean.addKalmanGain(ensemble.particles[0], observed_drifter_positions, innovations[0])
print("phi: ", phi_array)

# Plot the resulting Kalman gain
K_eta, K_hu, K_hv = ensemble.particles[0].download(interior_domain_only=True)

half_the_kalman_gain = ensemble.particles[0].small_scale_model_error.getRandomNumbers()

for field, title in zip([K_eta, K_hu, K_hv, half_the_kalman_gain], 
                        ["K_eta", "K_hu", "K_hv", "half_the_kalman_gain"]):
    fig = plt.figure(figsize=(4,4))
    ax = plt.subplot(1, 1, 1)
    plt.imshow(field, interpolation='None', origin='lower')
    plt.title(title)
    if field.shape[0] < sim_args["ny"]:
        ensemble._markDriftersInImshow(ax, observed_drifter_positions/sim_args["small_scale_perturbation_interpolation_factor"])
    else:
        ensemble._markDriftersInImshow(ax, observed_drifter_positions)
    plt.colorbar()
    


Sample perpendicular $\nu, \xi \sim N(0,I)$ 

In [ ]:
# Loop step 2: Sample xi \sim N(0, P), and get gamma in the process
gamma_array[0], zeta_array[0], std_norm_rand, std_norm_pend =\
    iewpfOcean.samplePerpendicular(ensemble.particles[0],
                                   return_original_random_numbers=True)
print("gamma: ", gamma_array)
print("zeta: ", zeta_array)

**Catch up** with the rest of the particles with the Kalman gain and perpendicular random fields

In [ ]:
for p in range(1, ensemble.getNumParticles()):
    # Pull particles towards observation by adding a Kalman gain term
    #     Also, we find phi within this function
    phi_array[p] = iewpfOcean.addKalmanGain(ensemble.particles[p], observed_drifter_positions, innovations[p])

    # Sample perpendicular xi and nu, and apply the SVD to both fields
    # Obtain gamma = xi^T * xi and nu^T * nu at the same time
    gamma_array[p], zeta_array[p] = iewpfOcean.samplePerpendicular(ensemble.particles[p])
 
    
c_array = phi_array + w_rest
print("phi_array:     ", phi_array)  
print("c_array:       ", c_array)
print("gamma_array:   ", gamma_array)
print("nu_norm_array: ", zeta_array)

Obtain **beta and target weight**
We don't use the information from the first artificial 

In [ ]:
use_zero_particle = False
if use_zero_particle:
    target_weight, beta = iewpfOcean.obtainTargetWeightTwoStage(c_array, 
                                                                zeta_array)
else:
    target_weight, beta = iewpfOcean.obtainTargetWeightTwoStage(c_array[1:],
                                                                zeta_array[1:])
print("target_weight: ", target_weight)
print("beta: ", beta)

**Solve implicit equation**

In [ ]:
# Loop step 3: Solve implicit equation
c_star = target_weight - c_array[0] - (beta - 1)*nu_norm_array[0]
alpha = iewpfOcean.solveImplicitEquation(gamma_array[0], target_weight, 
                                         w_rest[0], c_star, 
                                         particle_id=0)
print("c_star: ", c_star)
print("alpha:  ", alpha)


# Since the ocean field is sat to zero, we accept alpha=nan

Sample $\xi \sim N(0, P)$. With this function we only apply the SVD result to the standard normal distributed field.

In [ ]:
iewpfOcean.applySVDtoPerpendicular(ensemble.particles[0], observed_drifter_positions,
                                   alpha, beta)

# Transforming xi and nu 
axibetanu = np.sqrt(alpha)*std_norm_rand + np.sqrt(beta)*std_norm_pend


svd_rand = ensemble.particles[0].small_scale_model_error.getRandomNumbers()
svd_pend = ensemble.particles[0].small_scale_model_error.getPerpendicularRandomNumbers()

#imshow3(std_norm_rand, svd_rand, svd_rand-std_norm_rand,
#       title="xi N(0,I), xi N(0,SVD), xi N(0,SVD) - N(0,I)")
#imshow3(std_norm_pend, svd_pend, svd_pend-std_norm_pend,
#       title="nu N(0,I), nu N(0,SVD), nu N(0,SVD) - N(0,I)")

for field, title in zip([axibetanu, svd_rand, svd_rand-axibetanu], 
                        [r'$\sqrt{\alpha}\xi + \sqrt{\beta}\nu$', "SVD result", r'SVD result - $\sqrt{\alpha}\xi + \sqrt{\beta}\nu$)']):
    fig = plt.figure(figsize=(4,4))
    ax = plt.subplot(1, 1, 1)
    plt.imshow(field, interpolation='None', origin='lower')
    plt.title(title)
    ensemble._markDriftersInImshow(ax, observed_drifter_positions/sim_args["small_scale_perturbation_interpolation_factor"])
    plt.colorbar()

for field, title in zip([std_norm_pend, svd_pend, svd_pend-std_norm_pend], 
                        ["nu N(0,I)", "nu N(0,SVD)", "nu N(0,SVD) - N(0,I)"]):
    fig = plt.figure(figsize=(4,4))
    ax = plt.subplot(1, 1, 1)
    plt.imshow(field, interpolation='None', origin='lower')
    plt.title(title)
    ensemble._markDriftersInImshow(ax, observed_drifter_positions/sim_args["small_scale_perturbation_interpolation_factor"])
    plt.colorbar()

print("std_norm_rand dot std_norm_pend = ", np.sum(std_norm_rand*std_norm_pend))
print("std_norm_pend dot std_norm_pend = ", np.sum(std_norm_pend*std_norm_pend))
print("std_norm_rand dot std_norm_rand = ", np.sum(std_norm_rand*std_norm_rand))
print("")
print("svd_rand dot svd_pend = ", np.sum(svd_rand*svd_pend))
print("svd_pend dot svd_pend = ", np.sum(svd_pend*svd_pend))
print("svd_rand dot svd_rand = ", np.sum(svd_rand*svd_rand))


In [ ]:
# Add scaled sample from P to the state vector
ensemble.particles[0].small_scale_model_error.perturbSim(ensemble.particles[0],\
                                                         update_random_field=False, \
                                                         perturbation_scale=alpha,
                                                         perpendicular_scale=beta)  
# Plot the resulting Kalman gain
res_eta, res_hu, res_hv = ensemble.particles[0].download(interior_domain_only=True)
imshow3(res_eta, res_hu, res_hv, title="Results two-stage IEWPF")

In [ ]:
# Investigate how the xi sim N(0, P) was added to the solution

# From P^{1/2}* (a*xi + b*nu) ~ N(0, P):
eta_xi_P, hu_xi_P, hv_xi_P = np.zeros(dataShape), np.zeros(dataShape), np.zeros(dataShape)
ensemble.particles[0].small_scale_model_error.random_numbers_host = svd_rand
ensemble.particles[0].small_scale_model_error.perturbOceanStateCPU(eta_xi_P, hu_xi_P, hv_xi_P,
                                                                   sim_args["H"], sim_args["f"], 
                                                                   ghost_cells_x=2, ghost_cells_y=2,
                                                                   use_existing_CPU_random_numbers=True)
# From axibetanu N(0, Q):
eta_axibetanu_Q, hu_axibetanu_Q, hv_axibetanu_Q = np.zeros(dataShape), np.zeros(dataShape), np.zeros(dataShape)
ensemble.particles[0].small_scale_model_error.random_numbers_host = axibetanu
ensemble.particles[0].small_scale_model_error.perturbOceanStateCPU(eta_axibetanu_Q, hu_axibetanu_Q, hv_axibetanu_Q,
                                                                   sim_args["H"], sim_args["f"], 
                                                                   ghost_cells_x=2, ghost_cells_y=2,
                                                                   use_existing_CPU_random_numbers=True)

# From nu N(0, P):
#eta_nu_P, hu_nu_P, hv_nu_P = np.zeros(dataShape), np.zeros(dataShape), np.zeros(dataShape)
#ensemble.particles[0].small_scale_model_error.random_numbers_host = svd_pend
#ensemble.particles[0].small_scale_model_error.perturbOceanStateCPU(eta_nu_P, hu_nu_P, hv_nu_P,                                                                   sim_args["H"], sim_args["f"], 
#                                                                   ghost_cells_x=2, ghost_cells_y=2,
#                                                                   use_existing_CPU_random_numbers=True)
# From nu N(0, Q):
#eta_nu_Q, hu_nu_Q, hv_nu_Q = np.zeros(dataShape), np.zeros(dataShape), np.zeros(dataShape)
#ensemble.particles[0].small_scale_model_error.random_numbers_host = std_norm_pend
#ensemble.particles[0].small_scale_model_error.perturbOceanStateCPU(eta_nu_Q, hu_nu_Q, hv_nu_Q,                                                                   sim_args["H"], sim_args["f"], 
#                                                                   ghost_cells_x=2, ghost_cells_y=2,
#                                                                   use_existing_CPU_random_numbers=True)

imshow3(eta_axibetanu_Q, hu_axibetanu_Q, hv_axibetanu_Q, title="axibetanu Q", interior=True)
imshow3(eta_xi_P, hu_xi_P, hv_xi_P, title="xi P", interior=True)
imshow3(eta_xi_P-eta_axibetanu_Q, hu_xi_P-hu_axibetanu_Q, hv_xi_P-hv_axibetanu_Q, 
        title="xi P - axibetanu Q", interior=True)

#imshow3(eta_nu_Q, hu_nu_Q, hv_nu_Q, title="nu Q", interior=True)
#imshow3(eta_nu_P, hu_nu_P, hv_nu_P, title="nu P", interior=True)
#imshow3(eta_nu_P-eta_nu_Q, hv_nu_P-hu_nu_Q, hv_nu_P-hv_nu_Q, title="nu P - nu Q", interior=True)



**Build the result from the CPU array**

In [ ]:
print(K_eta.shape)
print(eta_xi_P.shape)
print(eta_nu_P.shape)
print(res_eta.shape)
print(eta_xi_P.shape)

use_beta = np.sqrt(beta) #np.sqrt(beta)
use_alpha = np.sqrt(alpha) #alpha

#eta_res_CPU = K_eta + use_beta*eta_nu_P[2:-2,2:-2] + use_alpha*eta_xi_P[2:-2,2:-2]
#hu_res_CPU  = K_hu  + use_beta* hu_nu_P[2:-2,2:-2] + use_alpha* hu_xi_P[2:-2,2:-2]
#hv_res_CPU  = K_hv  + use_beta* hv_nu_P[2:-2,2:-2] + use_alpha* hv_xi_P[2:-2,2:-2]
#eta_res_CPU = use_beta*eta_nu_P[2:-2,2:-2] + use_alpha*eta_xi_P[2:-2,2:-2]
#hu_res_CPU  = use_beta* hu_nu_P[2:-2,2:-2] + use_alpha* hu_xi_P[2:-2,2:-2]
#hv_res_CPU  = use_beta* hv_nu_P[2:-2,2:-2] + use_alpha* hv_xi_P[2:-2,2:-2]
eta_res_CPU = eta_xi_P[2:-2,2:-2]
hu_res_CPU  = hu_xi_P[2:-2,2:-2]
hv_res_CPU  = hv_xi_P[2:-2,2:-2]


res_eta_sub_K = res_eta - K_eta
res_hu_sub_K  = res_hu  - K_hu
res_hv_sub_K  = res_hv  - K_hv

imshow3(eta_res_CPU, hu_res_CPU, hv_res_CPU, title="results CPU")
#imshow3(res_eta, res_hu, res_hv, title="results GPU")
imshow3(res_eta_sub_K, res_hu_sub_K, res_hv_sub_K, title="results GPU")
imshow3(eta_res_CPU-res_eta_sub_K, hu_res_CPU-res_hu_sub_K, hv_res_CPU-res_hv_sub_K, title="results CPU - GPU")


In [ ]:
sim.logger.getEffectiveLevel()
